# HYPERPARAMETER TUNING

In [ ]:
import logging
import byron

In [2]:
byron.logger.setLevel(level=logging.CRITICAL)


In [3]:
NUM_BITS = 50


In [4]:
@byron.fitness_function
def fitness(genotype):
    """Parametric 1-max"""
    return sum(b == '1' for b in genotype)

In [5]:
macro = byron.f.macro('{v}', v=byron.f.array_parameter('01', NUM_BITS + 1))
top_frame = byron.f.sequence([macro])

In [6]:
evaluator = byron.evaluator.PythonEvaluator(fitness, strip_phenotypes=True)

## Default configuration

In [7]:
max_generation=500
lambda_=20
mu=10
top_n=5
lifespan=100
rewards=[0.7,0.3]
temperature = 0.85

In [8]:
# population = byron.ea.adaptive_ea(
#     top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
# )

## Tuning mu, lambda


In [9]:
#output_file = open('mu_lambda_mean_ind.txt', 'a')
#output_file = open('rewards_mean.txt', 'a')
#output_file = open('temperature_mean.txt', 'a')

In [10]:
from math import ceil
from byron.sys import get_operators
from time import time

In [11]:
def log_operators(f):
    all_ops = sorted(get_operators(), key=lambda o: (o.num_parents if o.num_parents is not None else -1, o.__name__))

    descr = {None: 'init', 1: 'mut', 2: 'xover'}
    ret = []
    #print(f"GENETIC OPERATORS\n")
    for op in all_ops:
        name = f'{op.__name__}'
        ret.append(f"{name}:{op.stats.calls};{op.stats.offspring};{op.stats.successes}")
    return ret

In [12]:
def lambda_all():
    for mu in range(10, 110, 10):
        output_file.write(f"mu: {mu}\n")
        print("mu: ", mu)
        for lambda_ in range (10,110,10):
            print("lambda: ", lambda_)
            mean_time = []

            #output_file.write(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")
            for _ in range(5):
                t1 = time()

                population = byron.ea.adaptive_ea(
                    top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
                )

                t2 = time()

                mean_time.append(t2-t1)
            output_file.write(f"time: {sum(mean_time)/len(mean_time)}\n")

In [13]:
def lambda_all_ind():
    for mu in range(10, 110, 10):
        output_file.write(f"mu: {mu}\n")
        print("mu: ", mu)
        for lambda_ in range (10,110,10):
            print("lambda: ", lambda_)
            mean_ind = []

            #output_file.write(f"MU VALUE: {mu} LAMBDA VALUE: {lambda_}\n")
            for _ in range(5):
               # t1 = time()

                population = byron.ea.adaptive_ea(
                    top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
                )

               # t2 = time()
                ind = population[0].describe(include_fitness=False, include_structure=False, include_age=True, include_lineage=False)
                t = ind.split(" ")[0].replace("𝐢", "")
                mean_ind.append(float(t))
            output_file.write(f"time: {sum(mean_ind)/len(mean_ind)}\n")

In [14]:
#lambda_all_ind()
# 20 10

## Rewards

In [15]:
from byron.registry import Statistics

In [16]:
def reset():
    all_ops = sorted(get_operators(), key=lambda o: (o.num_parents if o.num_parents is not None else -1, o.__name__))
    for op in all_ops:
        op.stats = Statistics()

In [17]:
def parse_info(info):
    values = []
    for i in info:
        name, stats = (i.split(":")[0], i.split(":")[1])
        values.append((float(stats.split(";")[0]),float(stats.split(";")[1]),float(stats.split(";")[2])))
    ret = [sum(v[0] for v in values), sum(v[1] for v in values), sum(v[2] for v in values)]
    return ret

In [18]:
# i = log_operators(1)
# print(i)
# c, o, s = parse_info(i)
# print(c,o,s)

In [19]:
def reward_all():
    population = None
    rewards = [0,0]
    for i in range(0,10):
        rewards[0] = i/10
        output_file.write(f"success: {rewards[0]}\n")
        print("success: ", rewards[0])
        for j in range (0,10):
            rewards[1] = j/10
            print("valid: ", rewards[1])
            mean_calls = []
            mean_offspring = []
            mean_successes = []
            mean_gen = []
            for _ in range(5):
                reset()
               # t1 = time()

                population = byron.ea.adaptive_ea(
                    top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
                )
                #t2 = time()
                info = log_operators(1)
                c,o,s = parse_info(info)
                mean_calls.append(c)
                mean_offspring.append(o)
                mean_successes.append(s)
                g = population[0].describe(include_fitness=False, include_structure=False, include_age=True, include_lineage=False)
                mean_gen.append(float(g.split(" ")[3]))
            output_file.write(f"ops:{sum(mean_calls)/len(mean_calls)};{sum(mean_offspring)/len(mean_offspring)};{sum(mean_successes)/len(mean_successes)};{sum(mean_gen)/len(mean_gen)}\n")
            print(population[0].describe(include_fitness=False, include_structure=False, include_age=True, include_lineage=False))
    return population

In [20]:
#pop =reward_all()

In [21]:
# g = pop[0].describe(include_fitness=False, include_structure=False, include_age=True, include_lineage=False)
# gg = g.split(" ")[3]
# print(gg)

## Temperature

In [22]:
def temperature_all():
    population = None
    rewards = [0,0]
    for i in range(50,100, 5):
            temperature = i/100
            mean_gen = []
            print(temperature)
            for _ in range(5):
                reset()
               # t1 = time()

                population = byron.ea.adaptive_ea(
                    top_frame, evaluator, max_generation=max_generation, lambda_=lambda_, mu=mu, max_fitness=NUM_BITS, top_n=top_n, lifespan=lifespan, rewards=rewards, temperature=temperature
                )
                #t2 = time()

                g = population[0].describe(include_fitness=False, include_structure=False, include_age=True, include_lineage=False)
                mean_gen.append(float(g.split(" ")[3]))
            output_file.write(f"gen:{sum(mean_gen)/len(mean_gen)}\n")
            print(population[0].describe(include_fitness=True, include_structure=False, include_age=True, include_lineage=False))
    return population

In [ ]:
#pop = temperature_all()

In [24]:
output_file.close()